In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from keras_tuner import HyperParameters

import autokeras as ak

In [2]:
from cerebro.nas.hphpmodel import HyperHyperModel

In [3]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node = ak.ClassificationHead(output_node1)

am = HyperHyperModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

In [4]:
am.graph

In [5]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

21/11/13 20:49:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
from cerebro.backend import SparkBackend
from cerebro.storage import LocalStore

backend = SparkBackend(spark_context=sc, num_workers=1)
store = LocalStore(prefix_path='/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments')

am.resource_bind(backend=backend, store=store)

CEREBRO => Time: 2021-11-13 20:49:38, Running 1 Workers


In [7]:
# cus_hp = HyperParameters()
# cus_hp.Fixed("learning_rate", value=0.0001)
# am.tuner_bind("hyperband", hyperparameters=cus_hp)

am.tuner_bind("gridsearch", hyperparameters=None)

INFO:tensorflow:Reloading Oracle from existing project ./test/oracle.json


In [8]:
tuner = am.tuner

tuner.search_space_summary()

Search space summary
Default search space size: 16
learning_rate (Fixed)
{'conditions': [], 'value': 0.0001}
conv_block_1/kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
conv_block_1/separable (Boolean)
{'default': False, 'conditions': []}
conv_block_1/max_pooling (Boolean)
{'default': True, 'conditions': []}
conv_block_1/dropout (Choice)
{'default': 0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
conv_block_1/num_blocks (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
conv_block_1/num_layers (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2], 'ordered': True}
conv_block_1/filters_0_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_0_1 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_1_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 3

In [10]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train = np.array(tf.one_hot(y_train, len(np.unique(y_train))))

In [11]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000, 10)


In [12]:
am.test_tuner_space(x=x_train, y=y_train)

In [13]:
tuner.search_space_summary()

Search space summary
Default search space size: 23
learning_rate (Fixed)
{'conditions': [], 'value': 0.0001}
conv_block_1/kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
conv_block_1/separable (Boolean)
{'default': False, 'conditions': []}
conv_block_1/max_pooling (Boolean)
{'default': True, 'conditions': []}
conv_block_1/dropout (Choice)
{'default': 0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
conv_block_1/num_blocks (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
conv_block_1/num_layers (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2], 'ordered': True}
conv_block_1/filters_0_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_0_1 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_1_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 3

In [14]:
tuner.oracle.hyperparameters.space

[Fixed(name: learning_rate, value: 0.0001),
 Choice(name: "conv_block_1/kernel_size", values: [3, 5, 7], ordered: True, default: 3),
 Boolean(name: "conv_block_1/separable", default: False),
 Boolean(name: "conv_block_1/max_pooling", default: True),
 Choice(name: "conv_block_1/dropout", values: [0.0, 0.25, 0.5], ordered: True, default: 0),
 Choice(name: "conv_block_1/num_blocks", values: [1, 2, 3], ordered: True, default: 2),
 Choice(name: "conv_block_1/num_layers", values: [1, 2], ordered: True, default: 2),
 Choice(name: "conv_block_1/filters_0_0", values: [16, 32, 64, 128, 256, 512], ordered: True, default: 32),
 Choice(name: "conv_block_1/filters_0_1", values: [16, 32, 64, 128, 256, 512], ordered: True, default: 32),
 Choice(name: "conv_block_1/filters_1_0", values: [16, 32, 64, 128, 256, 512], ordered: True, default: 32),
 Choice(name: "conv_block_1/filters_1_1", values: [16, 32, 64, 128, 256, 512], ordered: True, default: 32),
 Boolean(name: "res_net_block_1/pretrained", default:

In [15]:
tuner.oracle.init_search_space()

AttributeError: 'Fixed' object has no attribute 'sampling'

In [ ]:
test = HyperParameters()
test.Fixed("lr", value=0.01)
test.Choice("lr", values=[1,2,3])
test.Fixed("lr", value=0.02)

test.values

In [ ]:
trial = tuner.oracle.create_trial(tuner.tuner_id)
trial.summary()

In [ ]:
AutoKeras test

In [ ]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.Merge()([output_node1, output_node2])
output_node = ak.ClassificationHead()(output_node)

am = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

In [ ]:
dataset, validation_data = am._convert_to_dataset(
            x=x_train, y=y_train, validation_data=None, batch_size=32
        )

In [ ]:
dataset

In [ ]:
am._analyze_data(dataset)
am._build_hyper_pipeline(dataset)

In [ ]:
tuner = am.tuner

In [ ]:
tuner.hypermodel.hypermodel.set_fit_args(0.2, epochs=100)

In [ ]:
hp = tuner.oracle.get_space()

pipeline = tuner.hyper_pipeline.build(hp, dataset)
pipeline.fit(dataset)
dataset = pipeline.transform(dataset)

In [ ]:
dataset

In [ ]:
from autokeras.utils import data_utils
tuner.hypermodel.hypermodel.set_io_shapes(data_utils.dataset_shape(dataset))

In [ ]:
tuner.hypermodel.build(hp)
tuner.oracle.update_space(hp)

In [ ]:
tuner.search_space_summary()

In [ ]:
data_utils.dataset_shape(dataset)